In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd

In [2]:
# import sklearn

In [3]:
table_name='epa_'

### Load the Saved Data

In [4]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

### LODES industry is not useful- look up NAICS from wac instead
- TODO: do this in the script which prepares the USA data

In [5]:
import random
naics_cols=[col for col in zones.columns if 'naics' in col]
simpop_df['naics']=0
for ind, row in simpop_df.iterrows():
    if zones.loc[row['work_geoid'], 'emp_total']>0:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, weights=zones.loc[row['work_geoid'], naics_cols].values, k=1)[0]
    else:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, k=1)[0]

In [6]:
simpop_df=simpop_df.drop('industry', axis=1)

### Create the mobility system

In [7]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
getting external roads
	 Simplified from 2160 to 1619 edges and 1247 to 849 nodes
	 Pre-computing paths


In [8]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [9]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: DGGGPuQ22mcpfWjQXngg5SiLi4mZuMminrJbPKbtvWcc


In [10]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
                       mode_choice_model=Default_Mode_Choice_model(),
                       mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)

H.add_indicators([
    d, p, 
    m
])

# H.add_indicator(m)

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
Init simulation


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/CS_Indicators.py:619: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat


/opt/anaconda3/envs/cs/lib/python3.9/site-packages/osmnx/distance.py:426: UserWarning: The `get_nearest_nodes` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0


/opt/anaconda3/envs/cs/lib/python3.9/site-packages/brix/classes.py:981: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning


Dens Ind: 0:00:00.002291
https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
Prox Ind: 0:00:00.426055
Prox HM: 0:00:00.017140
https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.143109
	 trips post took 0:00:01.773500


In [11]:
# self=m
# import datetime
# print('Starting MM Update')
# start_calc=datetime.datetime.now()
# new_simpop=self.geogrid_updates(geogrid_data)
# new_simpop_df=pd.DataFrame(new_simpop)
# combined_simpop=self.base_simpop_df.append(new_simpop_df)
# sample_simpop_df=combined_simpop.sample(min(self.N_max, len(combined_simpop)))
# od_output, deckgl_trips, ind=self.simulate(sample_simpop_df)

In [12]:
geogrid_data=H.get_geogrid_data()
print(m.return_indicator(geogrid_data))
print(p.return_indicator(geogrid_data))
print(d.return_indicator(geogrid_data))

https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.224252
	 trips post took 0:00:02.382127
{'name': 'Active Mobility', 'raw_value': 0.156, 'value': 0.156, 'ref_value': 0.1, 'viz_type': 'radar'}
Prox Ind: 0:00:00.428824
Prox HM: 0:00:00.017773
{'heatmap': {'type': 'FeatureCollection', 'properties': ['housing', 'employment', 'healthcare', 'hospitality', 'shopping'], 'features': [{'type': 'Feature', 'properties': [0.13373043681382854, 0.11510874595094864, 0.045319465081723624, 0.07995226730310262, 0.13771929824561405], 'geometry': {'type': 'Point', 'coordinates': [-122.15591595904658, 37.469908656441945]}}, {'type': 'Feature', 'properties': [0.23557580004671805, 0.12714021286441463, 0.06463595839524518, 0.08711217183770883, 0.14035087719298245], 'geometry': {'type': 'Point', 'coordinates': [-122.15537676408951, 37.47015572900783]}}, {'type': 'Feature', 'properties': [0.23557580004671805, 0.12714021286441463, 0.06463595839524518, 0.0871121718

In [ ]:
H.listen()

Table URL: https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=epa_
Testing indicators
https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.009740
Prox Ind: 0:00:00.413617
Prox HM: 0:00:00.017159
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.106372
	 trips post took 0:00:02.171440
Performing initial update
Updating geogrid_data
https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
Updating table with hash: DGGGPuQ22mcpfWjQXngg5SiLi4mZuMminrJbPKbtvWcc
https://cityio.media.mit.edu/api/table/epa_/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.014084
Prox Ind: 0:00:00.507867
Prox HM: 0:00:00.016723
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.104065
	 trips post took 0:00:02.599488
Done with update
Local grid hash: DGGGPuQ22mcpfWjQXngg5SiLi4mZuMminrJbPKbtvWcc
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
Going into rest mode
Time active: 442.9099910259247
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/h

https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta

/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:04.031706
	 trips post took 0:00:01.787921
Done with update
Local grid hash: 7gGwNDs4vZzuTYfnWLZLerhYggSeLDqBB7NdWJgN71Gw
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa_/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/t